### GridsearchCV
performs hyperparameteropimization even on estimators that don't have a *CV version

In [10]:
from sklearn.datasets import load_wine

wine = load_wine()
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [11]:
from sklearn.model_selection import train_test_split

X, y = wine.data, wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)



### Pipeline 
Set up pipeline for execution inside gridsearchCV object

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
scaler = StandardScaler()

pipe_KNN = Pipeline([("scaler", scaler),("knn"), KNeighborsClassifier()])
pipe_log = make_pipeline(scaler, LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000))
pipe_forest = Pipeline([("forest", RandomForestClassifier())])

params_knn = {"knn__n_neighbors": list(range(1, 50))}
params_log = {'logisticregression__l1_ratio': np.linspace(0, 1, 20)}
params_forest = {'forest__n_estimators': list(range(50, 200, 10))}

classifier_KNN = GridSearchCV(estimator=pipe_KNN, param_grid=params_knn, cv=5, scoring='accuracy')
classifier_log = GridSearchCV(estimator=pipe_log, param_grid=params_log, cv=5, scoring='accuracy')
classifier_forest = GridSearchCV(estimator=pipe_forest, param_grid=params_forest, cv=5, scoring='accuracy')

#### Train classifiers on training data


In [13]:
classifier_KNN.fit(X_train, y_train)
classifier_log.fit(X_train, y_train)
classifier_forest.fit(X_train, y_train)

TypeError: 'KNeighborsClassifier' object is not subscriptable

In [ ]:
classifier_forest.best_estimator_.get_params()

In [ ]:
classifier_forest.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('forest', RandomForestClassifier())],
 'estimator__verbose': False,
 'estimator__forest': RandomForestClassifier(),
 'estimator__forest__bootstrap': True,
 'estimator__forest__ccp_alpha': 0.0,
 'estimator__forest__class_weight': None,
 'estimator__forest__criterion': 'gini',
 'estimator__forest__max_depth': None,
 'estimator__forest__max_features': 'sqrt',
 'estimator__forest__max_leaf_nodes': None,
 'estimator__forest__max_samples': None,
 'estimator__forest__min_impurity_decrease': 0.0,
 'estimator__forest__min_samples_leaf': 1,
 'estimator__forest__min_samples_split': 2,
 'estimator__forest__min_weight_fraction_leaf': 0.0,
 'estimator__forest__monotonic_cst': None,
 'estimator__forest__n_estimators': 100,
 'estimator__forest__n_jobs': None,
 'estimator__forest__oob_score': False,
 'estimator__forest__random_state': None,
 'estimator__forest__verbose': 0,
 'estimator__forest__warm_start': False,
 'estim

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
y_pred_knn = classifier_KNN.predict(X_test)

print(classification_report(y_test, y_pred_knn))
cm = confusion_matrix(y_test, y_pred_knn)
ConfusionMatrixDisplay.plot(cm)

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
y_pred_forest = classifier_forest.predict(X_test)

print(classification_report)

In [21]:
import joblib 

joblib.dump(classifier_log, 'production_model.joblib', compress=True)

my_model = joblib.load('production_model.joblib')

my_model.predict(X_test)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
'i